In [1]:
import cv2
import numpy as np
from pyzbar.pyzbar import decode
from pymongo import MongoClient
import logging
import tkinter as tk
from tkinter import messagebox

Setup logging

In [2]:
logging.basicConfig(level=logging.INFO)

Configuration

In [3]:
DB_URI = "mongodb://localhost:27017/"
DB_NAME = 'SMK'
COLLECTION_NAME = 'MoneyManager'

Connect to MongoDB

In [4]:
def connect_to_db():
    client = MongoClient(DB_URI)
    return client[DB_NAME]

Create or get collection

In [5]:
def get_collection(db):
    if COLLECTION_NAME in db.list_collection_names():
        logging.info('Collection found')
    else:
        db.create_collection(COLLECTION_NAME)
        logging.info('Collection created')
    return db[COLLECTION_NAME]

Scan QR Code

In [6]:
def scan_qr_code():
    cap = cv2.VideoCapture(0)
    cap.set(3, 640)
    cap.set(4, 480)
    b = []

    while len(b) < 1:
        ret, img = cap.read()
        cv2.imshow('QR Code Scanner', img)
        for barcode in decode(img):
            mydata = barcode.data.decode('utf-8')
            b.append(mydata)
        cv2.waitKey(1)

    cv2.destroyAllWindows()
    return b

Process scanned data

In [7]:
def process_data(data):
    l = data[0].split()
    res_str = [ele for ele in l if not ele.isdigit()]
    res_int = [int(ele) for ele in l if ele.isdigit()]
    return res_str, res_int

Insert data into database

In [8]:
def insert_data(collection, category, amount):
    record = {'Category': category, 'Amount': amount}
    collection.insert_one(record)
    logging.info(f"Inserted record: {record}")

Identify category and insert data

In [9]:
def categorize_and_insert(collection, res_str, res_int):
    category_keywords = {
        'Grocery': ['supermarket', 'food', 'market', 'delicatessen', 'hypermarket', 'greengrocery', 'pharmacy', 'grocery', 'store', 'greengrocer', 'general store', 'convenience store', 'walmart', 'mart', 'shop', 'grocer store', 'cashier', 'shopper', 'dairy', 'pizza', 'warehouse', 'mall', 'restaurants', 'marketplace', 'retailer'],
        'Food': ['restarunt', 'mess', 'cafe', 'inn', 'restaurant', 'motel', 'resort', 'hostel', 'resort hotel', 'hotel room', 'hyatt', 'mansion', 'beachfront'],
        'Transport': ['taxi', 'bus', 'train', 'flight', 'uber', 'lyft', 'cab', 'metro', 'subway', 'tram'],
        'Entertainment': ['cinema', 'movie', 'theater', 'concert', 'show', 'event', 'festival', 'amusement', 'park', 'zoo', 'aquarium'],
        'Utilities': ['electricity', 'water', 'gas', 'internet', 'phone', 'mobile', 'bill', 'utility'],
        'Rent': ['rent', 'lease', 'apartment', 'house', 'home', 'accommodation', 'lodging'],
        'Shopping': ['clothes', 'shoes', 'apparel', 'fashion', 'jewelry', 'accessories', 'electronics', 'gadgets', 'furniture'],
        'Healthcare': ['doctor', 'hospital', 'clinic', 'pharmacy', 'medication', 'drug', 'medicine', 'health', 'care', 'treatment', 'surgery'],
        'Education': ['school', 'college', 'university', 'tuition', 'course', 'class', 'training', 'education', 'study', 'learning'],
        'Others': ['miscellaneous', 'other', 'etc', 'misc']
    }

    for word in res_str:
        for category, keywords in category_keywords.items():
            if word in keywords:
                for amount in res_int:
                    insert_data(collection, category, amount)
                return

    # If no category is matched, categorize as 'Uncategorized'
    for amount in res_int:
        insert_data(collection, 'Uncategorized', amount)

Retrieve and calculate total expenditure

In [10]:
def calculate_total_expenditure(collection, category):
    x = collection.find({'Category': category}, {'Category': 1, 'Amount': 1})
    total = sum([data['Amount'] for data in x])
    logging.info(f"The total expenditure for {category} is {total}")
    return total

GUI Functions

In [11]:
def start_scanner():
    scanned_data = scan_qr_code()
    if not scanned_data:
        messagebox.showerror("Error", "No data scanned.")
        return

    res_str, res_int = process_data(scanned_data)
    categorize_and_insert(collection, res_str, res_int)
    messagebox.showinfo("Success", "Data scanned and saved successfully.")

def calculate_expenditure():
    category = category_entry.get()
    if not category:
        messagebox.showerror("Error", "Please enter a category.")
        return

    total_expenditure = calculate_total_expenditure(collection, category)
    result_label.config(text=f"The total expenditure for {category} is {total_expenditure}")

Main function

In [12]:
def main():
    global collection
    db = connect_to_db()
    collection = get_collection(db)

    # GUI Setup
    root = tk.Tk()
    root.title("QR Code Expense Tracker")

    scan_button = tk.Button(root, text="Start Scanner", command=start_scanner)
    scan_button.pack(pady=10)

    category_label = tk.Label(root, text="Enter Category:")
    category_label.pack(pady=5)

    global category_entry
    category_entry = tk.Entry(root)
    category_entry.pack(pady=5)

    calculate_button = tk.Button(root, text="Calculate Expenditure", command=calculate_expenditure)
    calculate_button.pack(pady=10)

    global result_label
    result_label = tk.Label(root, text="")
    result_label.pack(pady=10)

    root.mainloop()

if __name__ == "__main__":
    main()

INFO:root:Collection found
INFO:root:Inserted record: {'Category': 'Food', 'Amount': 143, '_id': ObjectId('6695a20a413fc9067a1f5697')}
INFO:root:The total expenditure for Food is 143
